In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import cv2
BATCH_SIZE = 32

trainGen = ImageDataGenerator(
    rescale= 1/255.,
    horizontal_flip=True,
    validation_split = 0.1
)

testGen = ImageDataGenerator(
    rescale= 1/255.,
)

train = trainGen.flow_from_directory(
    'C:\\Users\\ADMIN\\Desktop\\BITTU\\Verzeo AI\\LIVE MASK DETECTION\\dest_folder\\train', 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='training'
)

validation = trainGen.flow_from_directory(
    'C:\\Users\\ADMIN\\Desktop\\BITTU\\Verzeo AI\\LIVE MASK DETECTION\\dest_folder\\test', 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='validation'
)

Using TensorFlow backend.


Found 1185 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [2]:
import tensorflow as tf
import keras
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint

In [3]:
from keras.applications import MobileNetV2
mob = MobileNetV2(input_shape = (224,224,3),
    include_top = False,
    weights = 'imagenet')

In [4]:
mob.trainable = False

model = Sequential()
model.add(mob)
model.add(GlobalAveragePooling2D())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

checkpoint = ModelCheckpoint(
    'model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                81984     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 2,340,098
Trainable params: 82,114
Non-trainable params: 2,257,984
_________________________________________________________________


In [21]:
'''
hist = model.fit(
    train,
    epochs = 15,
    validation_data = validation,
    callbacks = [checkpoint]
)
'''

Epoch 1/15
38/38 [==============================] - 107s 3s/step - loss: 0.0756 - acc: 0.9696 - val_loss: 0.2284 - val_acc: 0.9444

Epoch 00001: val_loss did not improve from 0.03202
Epoch 2/15
38/38 [==============================] - 117s 3s/step - loss: 0.1452 - acc: 0.9527 - val_loss: 0.0845 - val_acc: 0.9444

Epoch 00002: val_loss did not improve from 0.03202
Epoch 3/15
38/38 [==============================] - 110s 3s/step - loss: 0.2157 - acc: 0.9494 - val_loss: 0.0440 - val_acc: 1.0000

Epoch 00003: val_loss did not improve from 0.03202
Epoch 4/15
38/38 [==============================] - 108s 3s/step - loss: 0.1615 - acc: 0.9502 - val_loss: 0.2067 - val_acc: 0.9444

Epoch 00004: val_loss did not improve from 0.03202
Epoch 5/15
38/38 [==============================] - 111s 3s/step - loss: 0.1587 - acc: 0.9637 - val_loss: 0.2313 - val_acc: 0.9444

Epoch 00005: val_loss did not improve from 0.03202
Epoch 6/15
38/38 [==============================] - 111s 3s/step - loss: 0.2216 - acc

In [5]:
model.load_weights('model.h5')

In [6]:
from keras.models import load_model
#model = load_model('model.h5')

resMap = {
        0 : 'Mask On',
        1 : 'Kindly Wear Mask'
    }

colorMap = {
        0 : (0,255,0),
        1 : (0,0,255)
    }

def prepImg(pth):
    return cv2.resize(pth,(224,224)).reshape(1,224,224,3)/255.0

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
while True:
    ret,img = cap.read()
    faces = classifier.detectMultiScale(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),1.1,2)

    for face in faces:
        slicedImg = img[face[1]:face[1]+face[3],face[0]:face[0]+face[2]]
        pred = model.predict(prepImg(img))
        pred = np.argmax(pred)

        cv2.rectangle(img,(face[0],face[1]),(face[0]+face[2],face[1]+face[3]),colorMap[pred],2)
        cv2.putText(img, resMap[pred],(face[0],face[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)        
        
                
    cv2.imshow('FaceMask Detection',img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()